In [16]:
import torch
import json
from tqdm import tqdm
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))
from dataset import Dataset
from utils import load_model, get_all_run_ids, get_device

In [28]:
models_path = "../trained/transformer-minidataset/top-runs"
run_ids = get_all_run_ids(path=models_path)
device = get_device()
models = {}
print("run_ids:", run_ids)
for _id in run_ids:
    models[_id] = load_model(_id, path=models_path, type="timelin").to(device)



run_ids: ['7jek9blg', 'xt1b5ncs', 'tbimy590', 'kdfu69md', 'ov2quu9v', '21soegq8', '0lhbjjox', '1qvcwtuy', 'b4gmn89t', 'jw8cws22', 'rarx6o1s', 'qfqaf3a0', '5qndphh8', 'a9wnfeis', 'awaen0te', '8wzblws2', '2s5lgg5h', 'e7pccu76', 'u9xd4jpq', '7wvj11dh', '56lvlmuj', 'gza0sltm', 'r21hwu9p', '7f4fvk0u', 'rlx8q7wd']


In [29]:
dataset = Dataset().load_dataset_from_pickle(pickle_path="../../dataset/data/0814/waveset_harmonics/processed.pkl")
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [30]:
max, min = torch.empty(0).to("cpu"), torch.empty(0).to("cpu")
models_range = {}

with torch.no_grad():
    for _id in run_ids:
        model = models[_id]
        
        _max, _min = torch.empty(0).to("cpu"), torch.empty(0).to("cpu")

        for batch_idx, (train_inputs, _) in enumerate(tqdm(dataloader)):
            out = model.forward_encoder(train_inputs.to(device)).detach().to("cpu")
            
            _max = torch.cat((_max, out.max(dim=1).values.max(dim=0).values.unsqueeze(0)), dim=0)
            _min = torch.cat((_min, out.min(dim=1).values.min(dim=0).values.unsqueeze(0)), dim=0)
            
            models_range[_id] = (_max, _min)
            
        _max = _max.max(dim=0).values     
        _min = _min.min(dim=0).values
               
        torch.cuda.empty_cache()
        
        models_range[_id] = {"min":_min.tolist(), "max": _max.tolist()}
        
        print("model", _id, "max", _max.tolist(), "min", _min.tolist())
            
        max = torch.cat((max, _max.unsqueeze(0)), dim=0)
        min = torch.cat((min, _min.unsqueeze(0)), dim=0)
        

100%|██████████| 202/202 [00:04<00:00, 41.08it/s]


model 7jek9blg max [1.4946842193603516, 1.562696099281311, 1.4944300651550293, 1.559212327003479] min [-1.5378073453903198, -1.457781434059143, -1.6402841806411743, -1.4616296291351318]


100%|██████████| 202/202 [00:10<00:00, 18.45it/s]


model xt1b5ncs max [0.5708659291267395, 0.8368782997131348, 0.7334136366844177, 0.9146367907524109] min [-1.0481925010681152, -0.7996190190315247, -0.7589512467384338, -0.5208720564842224]


100%|██████████| 202/202 [00:13<00:00, 15.18it/s]


...

model r21hwu9p max [1.6200934648513794, 1.1105892658233643, 1.121202826499939, 1.2507604360580444] min [-1.6600890159606934, -1.3960494995117188, -1.0675920248031616, -1.5820281505584717]


100%|██████████| 202/202 [00:05<00:00, 38.45it/s]


model 7f4fvk0u max [1.5808073282241821, 1.2837378978729248, 1.6427041292190552, 1.2102341651916504] min [-1.5337498188018799, -1.15310800075531, -1.4271023273468018, -1.6008830070495605]


100%|██████████| 202/202 [00:05<00:00, 37.09it/s]

model rlx8q7wd max [0.3706403076648712, 0.831834614276886, 0.9259507656097412, 1.4498611688613892] min [-0.8492431640625, -1.372462511062622, -1.1744530200958252, -0.6216397881507874]


In [31]:
models_range

{'7jek9blg': {'min': [-1.5378073453903198,
   -1.457781434059143,
   -1.6402841806411743,
   -1.4616296291351318],
  'max': [1.4946842193603516,
   1.562696099281311,
   1.4944300651550293,
   1.559212327003479]},
 'xt1b5ncs': {'min': [-1.0481925010681152,
   -0.7996190190315247,
   -0.7589512467384338,
   -0.5208720564842224],
  'max': [0.5708659291267395,
   0.8368782997131348,
   0.7334136366844177,
   0.9146367907524109]},
 'tbimy590': {'min': [-1.6370322704315186,
   -1.6363015174865723,
   -1.682663917541504,
   -1.468682050704956],
  'max': [1.5461868047714233,
   1.5116850137710571,
   1.6513432264328003,
   1.62351393699646]},
 'kdfu69md': {'min': [-1.5317035913467407,
   -1.4048943519592285,
   -1.5199394226074219,
   -1.4466338157653809],
  'max': [1.3213152885437012,
   1.5722863674163818,
   1.3613009452819824,
   1.3980050086975098]},
 'ov2quu9v': {'min': [-1.3095837831497192,
   -1.462934970855713,
   -1.1023536920547485,
   -1.4144636392593384],
  'max': [1.237347364425

In [32]:
type(models_range)

dict

In [34]:
with open("../../models/trained/transformer-minidataset/top-runs/models_range.json", "w") as f:
    json.dump(models_range, f)